In [1]:
from LLM import bots
from ASP_Scheduler.problem_descriptions import all_problems
from ASP_Scheduler import scheduler
from ASP_Scheduler import schedulerBaseline
import importlib
from pprint import pprint
from openai import OpenAI
import os



c:\Users\finna\GitHub\SchASPLM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# We illustrate the generation of a scheduling problem using an Inference client, in this case deepseek (using chain-of-thought and few-shot prompting)
# Notice you have to enter your api-key in line 132 of LLM/bots.py
# Notice that delays of 10 seconds have been added in scheduler.py to avoid reaching the token-limit of the inference client.

# Run the LLM scheduler
full_program = scheduler.full_ASP_program(
    all_problems['post_enrollment_based_course_time_tabling'],    # Input problem specifications for examination timetabling
    pipe=None,                                                    # Input the PIPEline object for the LLM
    printer=True,                                                 # Set to True to print intermediate outputs 
    k=5)                                                          # Number retries (0 means zero-shot)

# Print the full program as it is returned by the scheduler
print('----------------------------FULL PROGRAM----------------------------')
print(full_program)

Final response after 0 retries with 0 errors:
% Instance Template
event(Event).

room(Room, Capacity).

timeslot(Day, Hour).

student(Student, Event).

feature_requirement(Event, Feature).

room_feature(Room, Feature).

precedes(Event1, Event2).

allowed_timeslot(Event, Timeslot).
Errors: []
Instance Template:
% Instance Template
event(Event).

room(Room, Capacity).

timeslot(Day, Hour).

student(Student, Event).

feature_requirement(Event, Feature).

room_feature(Room, Feature).

precedes(Event1, Event2).

allowed_timeslot(Event, Timeslot).
Final response after 1 retries with 0 errors:
assigned(Event, Timeslot, Room) :- room(Room, _), timeslot(Timeslot), Event = #count{ E : assigned(E, Timeslot, Room) }.
Errors: []
Response:
% 1
N_events {{ assigned(Event, Timeslot, Room) : room(Room, _), timeslot(Timeslot) }} N_events :- event(Event).
Errors:
[{'type': 'MessageCode.RuntimeError', 'message': 'temp.lp:2:11-12: error: syntax error, unexpected {', 'line_number': 2, 'code': 'N_events {{ a

In [ ]:
# We illustrate the generation of a scheduling problem using a local model  (using chain-of-thought and few-shot prompting)
# Notice you have to have your api-key under .env


# If we are working locally, we just instantiate to None and by default we will call the llama 3.1 8B api
CHECKPOINT = None
# CHECKPOINT = "meta-llama/Llama-3.3-70B-Instruct"
# If we work on a cloud, we can use other models like this
PIPE = bots.load_pipe(model_checkpoint=CHECKPOINT, local_dir="./local_models", quantization_config='4bit')

# Examination Timetabling Example

# Run the LLM scheduler
full_program = scheduler.full_ASP_program(
    all_problems['post_enrollment_based_course_time_tabling'],    # Input problem specifications for examination timetabling
    pipe=PIPE,                                  # Input the PIPEline object for the LLM
    printer=False)                              # Set to True to print intermediate outputs           

# Print the full program as it is returned by the scheduler
print('----------------------------FULL PROGRAM----------------------------')
print(full_program)

In [ ]:
# We finally illustrate the generation of a scheduling problem using the baseline model.
# PIPE has to be configured as above

full_program = schedulerBaseline.full_ASP_program(
    all_problems['nurse_scheduling'], 
    pipe=PIPE, 
    printer=False)

print('----------------------------FULL PROGRAM----------------------------')
print(full_program)